In [92]:
from googleapiclient.discovery import build
api_key="<<YOUR API KEY>>"
yt = build('youtube','v3',developerKey=api_key) #check https://developers.google.com/docs/api/quickstart/python for more details
ID = '2P_5RE2DBXw' #id from the youtube video link

In [98]:
loop_comment=True
pageToken_comment=None
import time
import random
collect_comments={}
while loop_comment:
    data_comment=yt.commentThreads().list(part='snippet', videoId=ID, maxResults='100', pageToken=pageToken_comment,textFormat="plainText").execute()
    for i in data_comment['items']:
        comment = i["snippet"]['topLevelComment']["snippet"]["textDisplay"]
        likes = i["snippet"]['topLevelComment']["snippet"]['likeCount']
        totalReplyCount = i["snippet"]['totalReplyCount']
        collect_replies=None
        if totalReplyCount > 0:
            parent = i["snippet"]['topLevelComment']["id"]
            loop_replies=True
            pageToken_replies=None
            collect_replies=[]
            while loop_replies:
                data_replies = yt.comments().list(part='snippet', maxResults='100', pageToken=pageToken_replies,parentId=parent, textFormat="plainText").execute()
                for i in data_replies["items"]:
                    comment = i["snippet"]["textDisplay"]
                    likes = i["snippet"]['likeCount']
                    collect_replies.append((comment,likes))
                if data_replies.get('nextPageToken'):
                    pageToken_replies=data_replies["nextPageToken"]
                    print(pageToken_replies)
                else:
                    loop_replies=False
                
        if collect_replies:
            collect_comments[(comment,likes)]=collect_replies
        else:
            collect_comments[(comment,likes)]=[('','')]
    if data_comment.get('nextPageToken'):
        pageToken_comment=data_comment["nextPageToken"]
        print(pageToken_comment)
    else:
        loop_comment=False

In [95]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [96]:
df_collect=[]
for k,v in collect_comments.items():
    for val in v:
        df_collect.append((k[0],k[1],val[0],val[1]))

In [103]:
df=pd.DataFrame(df_collect,columns=['USER_COMMENT','COMMENT_LIKE','REPLY','REPLY_LIKE'])
df.to_html('sample.html',index=False,encoding='utf-16',justify='right',)